## XGBoost Regression
We use xgboost library and gridsearch cv to search for the best parameters.

In [ ]:
import xgboost as xgb
from sklearn import metrics
from xgboost import plot_importance
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
import hashlib
from xgboost import plot_importance, plot_tree
import math
from matplotlib import pyplot
import pandas as pd

In [ ]:
# index_ = ["years of warranty","better loan offer","well maintained","low fuel consumption","reg_date","power","engine_cap","mileage","no_of_owners","depreciation","coe","dereg_value","omv","arf","type_of_vehicle_bus/mini bus","type_of_vehicle_hatchback","type_of_vehicle_luxury sedan","type_of_vehicle_mid-sized sedan","type_of_vehicle_mpv","type_of_vehicle_others","type_of_vehicle_sports car","type_of_vehicle_stationwagon","type_of_vehicle_suv","type_of_vehicle_truck","type_of_vehicle_van","fuel_type_diesel","fuel_type_electric","fuel_type_petrol","fuel_type_petrol-electric","transmission","model_price", "is_new"]
index_ = ["low fuel consumption", "well maintained", "better loan offer","reg_date","power","engine_cap","mileage","no_of_owners","depreciation","coe","dereg_value","omv","arf","model_price", "is_new"]
#
temp_idx = ["acc_{}".format(i) for i in range(1, 12)]
temp_idx.extend(index_)

In [ ]:
#load the training set
def process_training_set(make_encoder, model_encoder):
    ori_data = pd.read_csv("../data/train_v6.csv")
    # ori_data['model'] = ori_data['model'].map(lambda x: x.lower())
    # ori_data['make'] = ori_data['make'].map(lambda x: x.lower())
    # ori_data['model'] = model_encoder.transform(list(ori_data['model'].values))
    # ori_data['make'] = make_encoder.transform(list(ori_data['make'].values))
    X = []
    y = []
    cnt = 0
    for index, row in ori_data.iterrows():
        if cnt == 0:
            cnt += 1
            continue
        temp_row = []
        # print(row["accessories_vectors"].replace('[', '').replace(']', '').replace(' ', '').split(','))
        acc_list = [1 if str(i) in row["accessories_vectors"].replace('[', '').replace(']', '').replace(' ', '').split(',') else 0 for i in range(1, 12)]
        temp_row.extend(acc_list)
        temp_row.extend(row[index_].tolist())
        # make_ = row["make"].lower()
        # model_ = row["model"].lower()
        # m = hashlib.md5()
        # m.update(make_.encode("utf-8"))
        # mo = hashlib.md5()
        # mo.update(model_.encode("utf-8"))
        # temp_row.append(int(m.hexdigest(), 16) % 500)
        # temp_row.append(int(mo.hexdigest(), 16) % 2000)
        # print(int(m.hexdigest(), 16))
        X.append(temp_row)
        y.append(float(row['price']))
    return X, y
    # train_features = pd.DataFrame(X)
    # train_features.to_csv("../data/train_features.csv")
    # train_labels = pd.DataFrame(y)
    # train_labels.to_csv("../data/train_labels.csv")
        # print(row["accessories_vectors"])

In [ ]:
def process_test_set(make_encoder, model_encoder):
    ori_data = pd.read_csv("../data/test_v6.csv")
    # ori_data['model'] = ori_data['model'].map(lambda x: x.lower())
    # ori_data['make'] = ori_data['make'].map(lambda x: x.lower())
    # ori_data['model'] = model_encoder.transform(list(ori_data['model'].values))
    # ori_data['make'] = make_encoder.transform(list(ori_data['make'].values))
    X = []
    id_list = []
    for index, row in ori_data.iterrows():
        temp_row = []
        # print(row["accessories_vectors"].replace('[', '').replace(']', '').replace(' ', '').split(','))
        acc_list = [1 if str(i) in row["accessories_vectors"].replace('[', '').replace(']', '').replace(' ', '').split(',') else 0 for i in range(1, 12)]
        temp_row.extend(acc_list)
        temp_row.extend(row[index_].tolist())
        # make_ = row["make"].lower()
        # model_ = row["model"].lower()
        # m = hashlib.md5()
        # m.update(make_.encode("utf-8"))
        # mo = hashlib.md5()
        # mo.update(model_.encode("utf-8"))
        # temp_row.append(int(m.hexdigest(), 16) % 500)
        # temp_row.append(int(mo.hexdigest(), 16) % 2000)
        # print(int(m.hexdigest(), 16))
        X.append(temp_row)
        id_list.append(row["listing_id"])
    return X, id_list

## Generate the datasets for training


In [ ]:
make_encoder, model_encoder = 0, 0
test_X, id_list = process_test_set(make_encoder, model_encoder)
x, y = process_training_set(make_encoder, model_encoder)
x, y = np.array(x), np.array(y)
print('x,y shape', np.array(x).shape, np.array(y).shape)

## Generate the model and predict the result for test set

In [ ]:
xgb_model = xgb.XGBRegressor(learning_rate=0.1,
        max_depth=5,
        min_child_weight=3,
        gamma=0,
        seed=27, tree_method='gpu_hist', subsample = 0.6, colsample_bytree=0.6, n_estimators=600, reg_alpha=0.1)
xgb_model.fit(x, y)
xgb.XGBRegressor.feature_names = temp_idx
xgb_model.feature_names = temp_idx
plot_importance(xgb_model)
# plot_tree(xgb_model)
# pyplot.savefig("xgboost_tree")
# pyplot.show()
# print(xgb_model.feature_importances_)
# pyplot.bar(range(len(xgb_model.feature_importances_)), xgb_model.feature_importances_)
pyplot.show()
y_pred = xgb_model.predict(np.array(test_X))
y_pred = np.around(y_pred, 1)
y_pred = y_pred.tolist()
t_id_list = [i for i in range(len(y_pred))]
res_df = pd.DataFrame({"Id":t_id_list, "Predicted":y_pred})
# res_df = res_df.drop_duplicates(subset=["Id"], keep="first")
res_df.to_csv("../data/submit.csv", index=None)

print("\tXGBoost模型：", cross_val_score(xgb_model, x, y, cv=5, scoring="neg_root_mean_squared_error").mean())

## Parameter adjustment par

In [ ]:
# subsample colsample_bytree
from sklearn.model_selection import GridSearchCV

param_test3 = {
    'subsample':[i/10.0 for i in range(6,10)],
'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch = GridSearchCV(
    estimator=xgb.XGBRegressor(
        learning_rate=0.1,
        max_depth=9,
        min_child_weight=3,
        n_estimators=500,
        gamma=0,
        seed=27, tree_method='gpu_hist'),
    param_grid=param_test3,
    scoring='neg_root_mean_squared_error',
    cv=5)
gsearch.fit(x, y)
print('subsample_colsample_bytree------------------')
print('gsearch1.grid_scores_', gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])
print('gsearch1.best_params_', gsearch.best_params_)
print('gsearch1.best_score_', gsearch.best_score_)

In [ ]:
# reg_alpha 
from sklearn.model_selection import GridSearchCV

param_test3 = {
    # 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
'reg_alpha':[0.1, 1]}
gsearch = GridSearchCV(
    estimator=xgb.XGBRegressor(
        learning_rate=0.1,
        max_depth=5,
        min_child_weight=3,
        gamma=0,
        seed=27, tree_method='gpu_hist', subsample = 0.6, colsample_bytree=0.6, n_estimators=600),
    param_grid=param_test3,
    scoring='neg_root_mean_squared_error',
    cv=5)
gsearch.fit(x, y)
print('reg_alpha------------------')
print('gsearch1.grid_scores_', gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])
print('gsearch1.best_params_', gsearch.best_params_)
print('gsearch1.best_score_', gsearch.best_score_)

In [ ]:
# max_depth 与 min_child_weight
param_test1 = {
    'max_depth': [i for i in range(3, 10, 2)],
    'min_child_weight': [i for i in range(1, 6, 2)]
}
# from sklearn import svm, grid_search, datasets
# from sklearn import grid_search
from sklearn.model_selection import GridSearchCV
gsearch = GridSearchCV(
    estimator=xgb.XGBRegressor(
        learning_rate=0.1,booster="gbtree",objective="reg:squarederror",n_estimators=500,reg_alpha=100,
        seed=27,tree_method='gpu_hist', colsample_bytree=0.6, subsample=0.6),
    param_grid=param_test1,
    scoring='neg_root_mean_squared_error',
    cv=5)
gsearch.fit(x, y)
print('max_depth_min_child_weight')
print('gsearch1.grid_scores_', gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])
print('gsearch1.best_params_', gsearch.best_params_)
print('gsearch1.best_score_', gsearch.best_score_)

In [ ]:
# gamma
param_test3 = {'gamma': [i / 10.0 for i in range(0, 20, 4)]}
from sklearn.model_selection import GridSearchCV

gsearch = GridSearchCV(
    estimator=xgb.XGBRegressor(
        learning_rate=0.1, booster="gbtree", objective="reg:squarederror", n_estimators=600,
        seed=27, tree_method='gpu_hist', max_depth=5, min_child_weight=3),
    param_grid=param_test3,
    scoring='neg_root_mean_squared_error',
    cv=5)
gsearch.fit(x, y)
print('max_depth_min_child_weight')
print('gsearch1.grid_scores_', gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])
print('gsearch1.best_params_', gsearch.best_params_)
print('gsearch1.best_score_', gsearch.best_score_)